# Sentiment analysis

In [ ]:
!pip install "torch==2.0.1" datasets torchinfo stanza flair transformers tweetnlp pysentimiento 1> /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.0.1 which is incompatible.


In [10]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import time
import gc
from textblob import Blobber
from textblob.sentiments import PatternAnalyzer, NaiveBayesAnalyzer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import torch
from torchinfo import summary
import stanza
from flair.nn import Classifier
from flair.data import Sentence
import datasets
import transformers
import tweetnlp
from pysentimiento import create_analyzer
import sys
import os
#from google.colab import drive

nltk.download('punkt', quiet=True)
nltk.download('movie_reviews', quiet=True)
nltk.download('vader_lexicon', quiet=True)
datasets.logging.disable_progress_bar()
transformers.logging.disable_progress_bar()
sys.stderr = open(os.devnull, 'w')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/korin.hoxha/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [33]:
INPUT_PATH = '/Users/korin.hoxha/Library/CloudStorage/OneDrive-RTLGroup/Korin Hoxha - Private/Personale/HU/SoSe 24/DEDA/content'  # Update this path as needed
OUTPUT_PATH = '/Users/korin.hoxha/Library/CloudStorage/OneDrive-RTLGroup/Korin Hoxha - Private/Personale/HU/SoSe 24/DEDA/model_output'
RAW_PATH = f'{OUTPUT_PATH}/raw'
FIGURE_PATH = 'figures'
DATASETS = ['reddit', 'youtube']
CATEGORIES = ['neg', 'pos']
BATCH_SIZE = 8
FONT_SIZE = 15
SENTENCES = ['This is a great test sentence with 10 tokens. ' * 10] * 10000
# EXAMPLES = [
#     'The movie was great',
#     'The movie was really great',
#     'The movie was not great',
#     'The movie was really not great',
#     'The movie was not that great',
#     'The movie could have been better'
# ]

In [22]:
def timed(func, *args, **kwargs):
    start = time.time()
    out = func(*args, **kwargs)
    return out, time.time() - start

def reduce_probabilities(a_prob):
    range_min = -1
    range_max = 1
    n_classes = len(a_prob)
    step = (range_max - range_min) / (n_classes - 1)
    a_reduced = 0
    for i in range(n_classes):
        a_reduced += (range_min + i * step) * a_prob[i]

    return a_reduced

def map_labels(a_prob):
    n_classes = len(a_prob)
    if n_classes == 2:
        class_to_label = {0: 'NEGATIVE', 1: 'POSITIVE'}
    elif n_classes == 3:
        class_to_label = {0: 'NEGATIVE', 1: 'NEUTRAL', 2: 'POSITIVE'}
    elif n_classes == 5:
        class_to_label = {0: 'NEGATIVE', 1: 'NEGATIVE', 2: 'NEUTRAL',
                          3: 'POSITIVE', 4: 'POSITIVE'}

    class_idx = np.argmax(a_prob)
    label = class_to_label[class_idx]
    return label

In [15]:
import os
print(os.getcwd())
os.chdir(os.getcwd())

/Users/korin.hoxha/Library/CloudStorage/OneDrive-RTLGroup/Korin Hoxha - Private/Personale/HU/SoSe 24/DEDA


# Datasets

In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Define INPUT_PATH and DATASETS
INPUT_PATH = '/Users/korin.hoxha/Library/CloudStorage/OneDrive-RTLGroup/Korin Hoxha - Private/Personale/HU/SoSe 24/DEDA/content'  # Update this path as needed
DATASETS = ['reddit', 'youtube']

def save_split(dataset):
    input_folder = os.path.join(INPUT_PATH)
    # Assuming the filenames for train and test sets are consistent, like 'neutrogena_reddit_output_train.csv' and 'neutrogena_reddit_output_test.csv'
    train_file = os.path.join(input_folder, f'neutrogena_{dataset}_output_train.csv')
    test_file = os.path.join(input_folder, f'neutrogena_{dataset}_output_test.csv')

    if os.path.exists(train_file) and os.path.exists(test_file):
        train_df = pd.read_csv(train_file)
        test_df = pd.read_csv(test_file)
    else:
        # If train and test files don't exist, read the main CSV file and split it into train and test sets
        main_file = os.path.join(input_folder, f'neutrogena_{dataset}_output1.csv')
        if os.path.exists(main_file):
            main_df = pd.read_csv(main_file)
            train_df, test_df = train_test_split(main_df, test_size=0.2, random_state=42)  # Adjust test_size as needed
        else:
            print(f"File {main_file} not found.")
            return

    # Save the train and test sets
    train_df.to_csv(os.path.join(input_folder, f'neutrogena_{dataset}_output_train.csv'), index=False)
    test_df.to_csv(os.path.join(input_folder, f'neutrogena_{dataset}_output_test.csv'), index=False)

for dataset in DATASETS:
    save_split(dataset)


In [15]:
df = pd.read_csv(INPUT_PATH+'/neutrogena_reddit_output_train.csv')
df1 = pd.read_csv(INPUT_PATH+'/neutrogena_youtube_output_train.csv')
df.head()
df1.head()

,Unnamed: 0,date,text,source,videoID,videoURL,language
0,723,2022-09-09,Hello ma’am I just got my neutrogena micro pol...,youtube,BlHhGeWoCFk,https://www.youtube.com/watch?v=BlHhGeWoCFk,en
1,192,2020-07-16,The neutrogena hydro boost “extra dry” is frag...,youtube,nRX9S6yLLaU,https://www.youtube.com/watch?v=nRX9S6yLLaU,en
2,948,2021-05-14,I was glad to see this review since I recently...,youtube,f61NqK3EoUc,https://www.youtube.com/watch?v=f61NqK3EoUc,en
3,568,2021-02-04,what do you think about the neutrogena water b...,youtube,Mw22-BSE1Og,https://www.youtube.com/watch?v=Mw22-BSE1Og,en
4,451,2020-02-04,I love the grapefruit wash from neutrogena. It...,youtube,nRX9S6yLLaU,https://www.youtube.com/watch?v=nRX9S6yLLaU,en


# Sentiment calculation

## TextBlob (Pattern, Naive Bayes)

In [5]:
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer, NaiveBayesAnalyzer

In [18]:
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer, NaiveBayesAnalyzer
import pandas as pd
import os

def load_textblob(model_name):
    if model_name == 'pattern':
        return PatternAnalyzer()
    elif model_name == 'naivebayes':
        return NaiveBayesAnalyzer()

def textblob_output(pipe, texts):
    output = [TextBlob(x, analyzer=pipe).sentiment._asdict() for x in texts]
    return output

def textblob_polarities(output, model_name):
    if model_name == 'pattern':
        polarities = [x['polarity'] for x in output]
    elif model_name == 'naivebayes':
        scores = [[x['p_neg'], x['p_pos']] for x in output]
        polarities = [reduce_probabilities(score) for score in scores]

    return polarities

def textblob_demo(model_name, digits):
    pipe = load_textblob(model_name)
    train_texts = read_train_texts()  # Function to read train texts from CSV files
    if train_texts:
        output = textblob_output(pipe, train_texts)
        polarities = textblob_polarities(output, model_name)
        for text, polarity in zip(train_texts, polarities):
            if polarity is not None:  # Check if polarity is not None before formatting
                print(f'{text:<35} {polarity:.{digits}f}')  # Adjusted to format the polarity properly
            else:
                print(f'{text:<35} Polarity not available')  # Print a message if polarity is None
        print('')
    else:
        print("No train texts found.")

def read_train_texts():
    train_texts = []
    # Assuming train CSV files are stored in a folder named 'train_sets'
    train_folder = '/Users/korin.hoxha/Library/CloudStorage/OneDrive-RTLGroup/Korin Hoxha - Private/Personale/HU/SoSe 24/DEDA/content'  # Update with the actual path
    for file in os.listdir(train_folder):
        if 'train' in file and file.endswith('.csv'):
            df = pd.read_csv(os.path.join(train_folder, file))
            train_texts.extend(df['text'])  # Replace 'text_column_name' with the actual column name containing texts
    return train_texts

def reduce_probabilities(score):
    # Function to reduce Naive Bayes probabilities to polarity
    # Implement as per your requirements
    pass

textblob_demo('pattern', 2)
textblob_demo('naivebayes', 5)


Hello ma’am I just got my neutrogena micro polish todayy and was searching on how to use this product and found your review! What a timing.. I am subscribed to your channel but its been long time you come up on my feed 🫣 anyway I’m so happy to ask you my question 😀 so how do I include this in my skincare? I usually follow a simple ctm ritual.. do I skip my cleanser once a week and use micro polish scrub instead? Pls reply 0.12
The neutrogena hydro boost “extra dry” is fragrance free! 0.14
I was glad to see this review since I recently tried both neutrogena and Olay.

I have ended up keeping the Olay Regenerist 24 as my night cream and three times a week I use the Olay retinol serum underneath the cream.  The other 4 nights of the week I use The Ordinary Niacinamide serum to help calm and heal. 0.09
what do you think about the neutrogena water boost water gel cleanser? 0.00
I love the grapefruit wash from neutrogena. It's like the only thing that works for me and I've been using it for 

In [23]:
!python3 -m textblob.download_corpora

In [24]:
import nltk
import numpy as np

for category in CATEGORIES:
    word_list = nltk.corpus.movie_reviews.words(categories=category)
    not_count = np.sum([1 if word == 'not' else 0 for word in word_list])
    print(f'{category}: {not_count}')

neg: 2651
pos: 2926


In [23]:
import numpy as np
import time  

def textblob_time(model_name, n):
    pipe = load_textblob(model_name)
    t_list = []
    for i in range(n):
        _, t = timed(textblob_output, pipe, SENTENCES)
        t_list.append(t)

    t_average = np.round(np.mean(t_list), 1)
    t_rounds = np.array2string(np.round(t_list, 1), separator=', ')
    print(f'{model_name}: {t_average} s (average of {n} rounds), {t_rounds} s')

textblob_time('pattern', 3)
textblob_time('naivebayes', 3)


pattern: 3.1 s (average of 3 rounds), [3.2, 3.1, 3.1] s
naivebayes: 6.9 s (average of 3 rounds), [8.2, 5.8, 6.6] s


In [37]:
import json
def textblob_raw():
    for model_name in ['pattern', 'naivebayes']:
        pipe = load_textblob(model_name)
        for dataset in DATASETS:
            #for category in CATEGORIES:
                df_text = pd.read_csv(f'{INPUT_PATH}/neutrogena_{dataset}_output_train.csv')
                texts = df_text['text'].to_list()
                output, t = timed(textblob_output, pipe, texts)
                output_json = [json.dumps(x) for x in output]
                df_raw = pd.DataFrame({f'{model_name}_output': output_json})
                df_raw.to_csv(f'{RAW_PATH}/textblob_{model_name}_neutrogena_{dataset}.txt', index=False)
                print(f'{model_name}, {dataset}: {np.round(t, 1)} s')

        print('')

textblob_raw()

# CPU: pattern -> 11-14 s, naivebayes -> 20-33 s

pattern, reddit: 0.0 s
pattern, youtube: 0.1 s

naivebayes, reddit: 2.6 s
naivebayes, youtube: 0.2 s



In [39]:
def textblob_processed():
    for dataset in DATASETS:
        for category in CATEGORIES:
            df_processed = pd.DataFrame()
            for model_name in ['pattern', 'naivebayes']:
                df_raw = pd.read_csv(f'{RAW_PATH}/textblob_{model_name}_neutrogena_{dataset}.txt')
                output = df_raw[f'{model_name}_output'].apply(lambda x: json.loads(x)).to_list()
                polarities = textblob_polarities(output, model_name)
                df_processed[f'textblob_{model_name}_polarity'] = polarities

            df_processed.to_csv(f'{OUTPUT_PATH}/textblob_{dataset}.txt', index=False)

textblob_processed()

## NLTK VADER

In [40]:
def load_vader():
    pipe = SentimentIntensityAnalyzer()
    return pipe

In [30]:
import nltk
nltk.download('vader_lexicon')

True

In [41]:
import os
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def load_vader():
    pipe = SentimentIntensityAnalyzer()
    return pipe

def vader_output(pipe, texts):
    output = [pipe.polarity_scores(x) for x in texts]
    return output

def vader_polarities(output):
    polarities = [x['compound'] for x in output]
    return polarities

def vader_demo(digits):
    pipe = load_vader()
    train_texts = read_train_texts()  # Function to read train texts from CSV files
    output = vader_output(pipe, train_texts)
    polarities = vader_polarities(output)
    for text, polarity in zip(train_texts, polarities):
        print(f'{text:<35} {polarity:.{digits}f}')

def read_train_texts():
    train_texts = []
    OUTPUT_PATH = '/Users/korin.hoxha/Library/CloudStorage/OneDrive-RTLGroup/Korin Hoxha - Private/Personale/HU/SoSe 24/DEDA/model_output'
    for file in os.listdir(OUTPUT_PATH):
        if 'train' in file and file.endswith('.csv'):
            df = pd.read_csv(os.path.join(OUTPUT_PATH, file))
            train_texts.extend(df['text'])  # Assuming 'text' is the column containing text data
    return train_texts

vader_demo(2)

In [42]:
def vader_time(n):
    pipe = load_vader()
    t_list = []
    for i in range(n):
        _, t = timed(vader_output, pipe, SENTENCES)
        t_list.append(t)

    t_average = np.round(np.mean(t_list), 1)
    t_rounds = np.array2string(np.round(t_list, 1), separator=', ')
    print(f'vader: {t_average} s (average of {n} rounds), {t_rounds} s')

vader_time(3)

# CPU: vader -> 5 s

vader: 2.2 s (average of 3 rounds), [2. , 2.1, 2.4] s


In [47]:
def vader_raw():
    pipe = load_vader()
    for dataset in DATASETS:
            df_text = pd.read_csv(f'{INPUT_PATH}/neutrogena_{dataset}_output_train.csv')
            texts = df_text['text'].to_list()
            output, t = timed(vader_output, pipe, texts)
            output_json = [json.dumps(x) for x in output]
            df_raw = pd.DataFrame({'vader_output': output_json})
            df_raw.to_csv(f'{RAW_PATH}/vader_raw_{dataset}.csv', index=False)
            print(f'vader, {dataset}: {np.round(t, 1)} s')

vader_raw()

# CPU: vader -> 16-21 s

vader, reddit: 0.0 s
vader, youtube: 0.2 s


In [49]:
def vader_processed():
    for dataset in DATASETS:
            df_raw = pd.read_csv(f'{RAW_PATH}/vader_raw_{dataset}.csv')
            output = df_raw['vader_output'].apply(lambda x: json.loads(x)).to_list()
            polarities = vader_polarities(output)
            df_processed = pd.DataFrame({'vader_polarity': polarities})
            df_processed.to_csv(f'{OUTPUT_PATH}/vader_{dataset}.csv', index=False)

vader_processed()

## Flair (RNN, distilBERT)

In [50]:
def load_flair(model_name):
    if model_name == 'rnn':
        pipe = Classifier.load('sentiment-fast')
    elif model_name == 'distilbert':
        pipe = Classifier.load('sentiment')

    return pipe

In [51]:
from flair.nn import Classifier

def flair_summary(model_name):
    model = load_flair(model_name)
    print(model)
    print(summary(model))
    print('')
    del model
    gc.collect()
    torch.cuda.empty_cache()

flair_summary('rnn')
flair_summary('distilbert')

TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('en-crawl')
    )
    (word_reprojection_map): Linear(in_features=300, out_features=300, bias=True)
    (rnn): LSTM(300, 256, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): BCEWithLogitsLoss()
  (weights): None
  (weight_tensor) None
)


NameError: name 'summary' is not defined

In [41]:
from flair.data import Sentence

def flair_output(pipe, texts):
    sentences = [Sentence(text) for text in texts]
    pipe.predict(sentences, mini_batch_size=BATCH_SIZE)
    output = []
    for sentence in sentences:
        result = sentence.labels[0]
        label = result.value
        score = result.score
        output.append({label: score})

    return output

def flair_polarities(output):
    scores = []
    for x in output:
        if 'NEGATIVE' in x.keys():
            score = [x['NEGATIVE'], (1 - x['NEGATIVE'])]
        elif 'POSITIVE' in x.keys():
            score = [(1 - x['POSITIVE']), x['POSITIVE']]
        scores.append(score)

    polarities = [reduce_probabilities(score) for score in scores]
    labels = [map_labels(score) for score in scores]
    return polarities, labels

def flair_demo(model_name, digits):
    pipe = load_flair(model_name)
    output = flair_output(pipe, EXAMPLES)
    polarities, _ = flair_polarities(output)
    for text, polarity in zip(EXAMPLES, polarities):
        print(f'{text:<35} {polarity:.{digits}}')

    print('')
    del pipe
    gc.collect()
    torch.cuda.empty_cache()

flair_demo('rnn', 5)
flair_demo('distilbert', 5)

The movie was great                 0.84534
The movie was really great          0.7819
The movie was not great             -0.99959
The movie was really not great      -0.99843
The movie was not that great        -0.9996
The movie could have been better    -0.94291

The movie was great                 0.90476
The movie was really great          0.9369
The movie was not great             -0.99895
The movie was really not great      -0.99819
The movie was not that great        -0.99941
The movie could have been better    -0.99972



In [42]:
def flair_time(model_name, n):
    pipe = load_flair(model_name)
    t_list = []
    for i in range(n):
        _, t = timed(flair_output, pipe, SENTENCES)
        t_list.append(t)

    t_average = np.round(np.mean(t_list), 1)
    t_rounds = np.array2string(np.round(t_list, 1), separator=', ')
    print(f'{model_name}: {t_average} s (average of {n} rounds), {t_rounds} s')
    del pipe
    gc.collect()
    torch.cuda.empty_cache()

flair_time('rnn', 3)
flair_time('distilbert', 3)

# GPU: rnn -> 40 s, distilbert -> 60 s

rnn: 23.9 s (average of 3 rounds), [22.2, 26.5, 23. ] s
distilbert: 204.9 s (average of 3 rounds), [209.5, 203.4, 201.8] s


In [43]:
def flair_raw():
    for model_name in ['rnn', 'distilbert']:
        pipe = load_flair(model_name)
        for dataset in DATASETS:
            for category in CATEGORIES:
                df_text = pd.read_csv(f'{INPUT_PATH}/{dataset}_{category}.txt')
                texts = df_text['text'].to_list()
                output, t = timed(flair_output, pipe, texts)
                output_json = [json.dumps(x) for x in output]
                df_raw = pd.DataFrame({f'{model_name}_output': output_json})
                df_raw.to_csv(f'{RAW_PATH}/flair_{model_name}_{dataset}_{category}.txt', index=False)
                print(f'{model_name}, {dataset}_{category}: {np.round(t, 1)} s')

        print('')
        del pipe
        gc.collect()
        torch.cuda.empty_cache()

flair_raw()

# GPU: rnn -> 45-65 s, distilbert -> 85-105 s

rnn, yelp_neg: 39.7 s
rnn, yelp_pos: 33.4 s
rnn, tweet_neg: 3.5 s
rnn, tweet_pos: 1.2 s
rnn, finance_neg: 0.3 s
rnn, finance_pos: 1.7 s

distilbert, yelp_neg: 381.9 s
distilbert, yelp_pos: 293.4 s
distilbert, tweet_neg: 30.1 s
distilbert, tweet_pos: 16.5 s
distilbert, finance_neg: 4.6 s
distilbert, finance_pos: 10.2 s



In [44]:
def flair_processed():
    for dataset in DATASETS:
        for category in CATEGORIES:
            df_processed = pd.DataFrame()
            for model_name in ['rnn', 'distilbert']:
                df_raw = pd.read_csv(f'{RAW_PATH}/flair_{model_name}_{dataset}_{category}.txt')
                output = df_raw[f'{model_name}_output'].apply(lambda x: json.loads(x)).to_list()
                polarities, labels = flair_polarities(output)
                df_processed[f'flair_{model_name}_polarity'] = polarities
                df_processed[f'flair_{model_name}_label'] = labels

            df_processed.to_csv(f'{OUTPUT_PATH}/flair_{dataset}_{category}.txt', index=False)

flair_processed()

# Sentiment classification


## Classification results

In [2]:
def combine_all():
    packages = ['textblob', 'vader', 'flair'
                ]
    for dataset in DATASETS:
        #for category in CATEGORIES:
            df_all = pd.DataFrame()
            for package in packages:
                df = pd.read_csv(f'{OUTPUT_PATH}/{package}_{dataset}_{category}.txt')
                df_all = pd.concat([df_all, df], axis=1)

            df_all.to_csv(f'{OUTPUT_PATH}/all_{dataset}_{category}.txt', index=False)

combine_all()

In [3]:
model_title = (
    ('textblob_pattern', 'TextBlob Pattern'),
    ('textblob_naivebayes', 'TextBlob Naive Bayes'),
    ('vader', 'VADER'),
    ('flair_rnn', 'Flair RNN'),
    ('flair_distilbert', 'Flair DistilBERT')
    )

In [ ]:
def classification_polarity_pos_neg(model_name, dataset, threshold=None):
    df_pos = pd.read_csv(f'{OUTPUT_PATH}/all_{dataset}_pos.txt')
    df_neg = pd.read_csv(f'{OUTPUT_PATH}/all_{dataset}_neg.txt')
    if threshold != None:
        col = f'{model_name}_polarity'
        tp = np.double(df_pos.query(f'{col} > {threshold}').shape[0])
        fp = np.double(df_neg.query(f'{col} > {threshold}').shape[0])
        tn = np.double(df_neg.query(f'{col} < -{threshold}').shape[0])
        fn = np.double(df_pos.query(f'{col} < -{threshold}').shape[0])
    else:
        col = f'{model_name}_label'
        tp = np.double(df_pos.query(f'{col} == "POSITIVE"').shape[0])
        fp = np.double(df_neg.query(f'{col} == "POSITIVE"').shape[0])
        tn = np.double(df_neg.query(f'{col} == "NEGATIVE"').shape[0])
        fn = np.double(df_pos.query(f'{col} == "NEGATIVE"').shape[0])

    accuracy = np.round((tp + tn) / (tp + tn + fp + fn), 2)
    ratio = np.round((tp + tn + fp + fn) / (df_pos.shape[0] + df_neg.shape[0]), 2)
    precision_pos = tp / (tp + fp)
    precision_neg = tn / (tn + fn)
    recall_pos = tp / (tp + fn)
    recall_neg = tn / (tn + fp)
    f1_pos = 2 * (precision_pos * recall_pos) / (precision_pos + recall_pos)
    f1_neg = 2 * (precision_neg * recall_neg) / (precision_neg + recall_neg)
    metrics_pos = np.round([precision_pos, recall_pos, f1_pos, accuracy], 2)
    metrics_neg = np.round([precision_neg, recall_neg, f1_neg], 2)
    return accuracy, metrics_pos, metrics_neg, ratio

for model_name, title in model_title:
    for dataset in DATASETS:
        threshold = 0
        accuracy, metrics_pos, metrics_neg, ratio = classification_polarity_pos_neg(model_name, dataset, threshold)
        print(f'{title} - {dataset} (threshold {threshold}):')
        print(f'- Accuracy {accuracy}, Ratio {ratio}')
        print(f'- Negative class: Precision {metrics_neg[0]}, Recall {metrics_neg[1]}, F1 {metrics_neg[2]}')
        print(f'- Positive class: Precision {metrics_pos[0]}, Recall {metrics_pos[1]}, F1 {metrics_pos[2]}\n')
        if model_name in ['huggingface_bert']:
            accuracy, metrics_pos, metrics_neg, ratio = classification_polarity_pos_neg(model_name, dataset)
            print(f'{title} - {dataset} (maximum probability):')
            print(f'- Accuracy {accuracy}, Ratio {ratio}')
            print(f'- Negative class: Precision {metrics_neg[0]}, Recall {metrics_neg[1]}, F1 {metrics_neg[2]}')
            print(f'- Positive class: Precision {metrics_pos[0]}, Recall {metrics_pos[1]}, F1 {metrics_pos[2]}\n')

## Sentiment histograms


In [4]:
def histogram(model_name, title, dataset, ax, type='polarity'):
    file_to_name = {
        'neg': 'Negative',
        'pos': 'Positive',
        'obj': 'Objective',
        'subj': 'Subjective'
    }
    if type == 'polarity':
        bins = np.arange(-1.05, 1.15, 0.1)
        file1 = 'neg'
        file2 = 'pos'
    elif type == 'subjectivity':
        bins = np.arange(0, 1.05, 0.05)
        file1 = 'obj'
        file2 = 'subj'

    col = f'{model_name}_{type}'
    df1 = pd.read_csv(f'{OUTPUT_PATH}/all_{dataset}_{file1}.txt')
    df2 = pd.read_csv(f'{OUTPUT_PATH}/all_{dataset}_{file2}.txt')
    df1[col].plot(kind='hist', bins=bins, alpha=0.5, ax=ax)
    df2[col].plot(kind='hist', bins=bins, alpha=0.5, ax=ax)
    mean1 = df1[col].mean()
    mean2 = df2[col].mean()
    name1 = file_to_name[file1]
    name2 = file_to_name[file2]
    ax.legend([f'{name1} ({mean1:.2g})', f'{name2} ({mean2:.2g})'], loc='upper left')
    ax.set_title(f'{title} - {dataset}')
    ax.set_xlabel(type.capitalize())

In [5]:
def accuracy_ratio(model_name, title, dataset, ax):
    thresholds = np.arange(0, 1, 0.05)
    accuracies = []
    ratios = []
    for threshold in thresholds:
        accuracy, _, _, ratio = classification_polarity_pos_neg(model_name, dataset, threshold)
        accuracies.append(accuracy)
        ratios.append(ratio)

    ax.plot(thresholds, accuracies)
    ax.plot(thresholds, ratios)
    ax.set_title(f'{title} - {dataset}')
    ax.legend(['Accuracy (neg/pos)', 'Ratio (neg/pos vs all)'], loc='best')
    ax.set_xlabel('Polarity threshold')
    plt.tight_layout(rect=(0.25, 0, 0.75, 1))
    plt.grid()

In [6]:
def plot_all(model_name, title, dataset):
    fig, ax = plt.subplots(1, 2, figsize=(20, 4))
    histogram(model_name, title, dataset, ax[0])
    ax[0].set_ylabel('Count')
    accuracy_ratio(model_name, title, dataset, ax[1])
    plt.savefig(f'{FIGURE_PATH}/{model_name}_{dataset}.png', bbox_inches='tight')
    plt.show()

In [8]:
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({
    'axes.titlesize': FONT_SIZE + 2,
    'axes.labelsize': FONT_SIZE,
    'xtick.labelsize': FONT_SIZE,
    'ytick.labelsize': FONT_SIZE,
    'legend.fontsize': FONT_SIZE - 2,
    'font.size': FONT_SIZE,
})
np.seterr(divide='ignore', invalid='ignore')

for model_name, title in model_title:
    for dataset in DATASETS:
        plot_all(model_name, title, dataset)

## Prediction definitions


In [10]:
threshold = 0.2
height = 1
fig, ax = plt.subplots(1, 1, figsize=(9, 4))
ax.set_title('Predicted classes')
ax.set_xlabel('Polarity')
ax.set_xticks(np.arange(-1.0, 1.5, 0.5))
ax.set_yticks([])
ax.set_xlim([-1.1, 1.1])
ax.set_ylim([0, height])
rect_neg = matplotlib.patches.Rectangle((-1, 0), (1 - threshold), height, color='red', alpha=0.2)
rect_neu = matplotlib.patches.Rectangle((-threshold, 0), 2 * threshold, height, color='black', alpha=0.2)
rect_pos = matplotlib.patches.Rectangle((threshold, 0), (1 - threshold), height, color='green', alpha=0.2)
ax.add_patch(rect_neg)
ax.add_patch(rect_neu)
ax.add_patch(rect_pos)
ax.text(-0.65, 0.4 * height, 'Negative', rotation=90)
ax.text(-0.05, 0.4 * height, 'Neutral', rotation=90)
ax.text(0.55, 0.4 * height, 'Positive', rotation=90)
ax.text(-threshold - 0.2, 0.1 * height, '-t', style='italic')
ax.text(threshold + 0.05, 0.1 * height, 't', style='italic')
plt.tight_layout(rect=(0.25, 0, 0.75, 1))
plt.savefig(f'{FIGURE_PATH}/classes_polarity.png')
plt.show()